In [ ]:
#clone the repo so you can mess around with it:
!git clone https://github.com/apapiu/guided-diffusion-keras.git
! pip install git+https://github.com/openai/CLIP.git

#this is where the data is:
!git lfs install
!git clone https://huggingface.co/apapiu/diffusion_model_aesthetic_keras
! pip3 install tensorflow==2.13.0
import os
import sys
import numpy as np
import pandas as pd

from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10, cifar100
from keras.utils.vis_utils import plot_model

from tensorflow import keras
from matplotlib import pyplot as plt

####local imports:
sys.path.append("/content/guided-diffusion-keras/guided_diffusion")
from denoiser import get_network
from utils import batch_generator, plot_images, get_data, preprocess


In [ ]:
#########
# CONFIG:
#########

train_model=True

image_size = 64
num_channels = 3
epochs = 50
class_guidance = 4

# architecture
channels = 96
channel_multiplier = [1, 2, 3, 4]
block_depth = 2
emb_size = 512  # CLIP/label embedding
num_classes = 12  # only used if precomputed_embedding=False
attention_levels = [0, 0, 1, 0]

embedding_dims = 32
embedding_max_frequency = 1000.0

precomputed_embedding = True
save_in_drive = False
widths = [c * channels for c in channel_multiplier]

###train process config:
batch_size = 64
num_imgs = 36 #num imgs to test on - should be a square - 25, 64, 100 etc.
row = int(np.sqrt(num_imgs))

validation_num = 100 #don't train on first validation_num images so we can use as visual validation
train_size = 40000 #only train on a subset of train data - do this for faster results..

learning_rate = 0.0003

MODEL_NAME = "model_test_aesthetic"
from_scratch = False #if False will load model from model path and continue training
file_name = "from_huggingface"

data_dir = '/content/diffusion_model_aesthetic_keras'
captions_path = os.path.join(data_dir, "captions.csv")
imgs_path = os.path.join(data_dir, "imgs.npy")
embedding_path = os.path.join(data_dir, "embeddings.npy")

if save_in_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/'
    home_dir = os.path.join(drive_path, MODEL_NAME)
else:
    home_dir = MODEL_NAME

if not os.path.exists(home_dir):
    os.mkdir(home_dir)

model_path = os.path.join(home_dir, MODEL_NAME + ".h5")

In [ ]:
import tensorflow as tf
print(model_path,  tf. __version__)

In [ ]:
##################################
###########Loading Data And Model:
##################################

#load the data from a numpy file file:
captions = pd.read_csv(captions_path)
train_data, train_label_embeddings = np.load(imgs_path), np.load(embedding_path)
#train_data, train_label_embeddings = get_data(npz_file_name=file_name, prop=0.6, captions=False)
print(train_data.shape)

labels = train_label_embeddings[:num_imgs]

np.random.seed(100)
rand_image = np.random.normal(0, 1, (num_imgs, image_size, image_size, num_channels))

if from_scratch:
    autoencoder = get_network(image_size,
                              widths,
                              block_depth,
                              num_classes=num_classes,
                              attention_levels=attention_levels,
                              emb_size=emb_size,
                              num_channels=num_channels,
                              precomputed_embedding=precomputed_embedding)

    autoencoder.compile(optimizer="adam", loss="mae")
else:
    autoencoder = keras.models.load_model(model_path)

##################
#Some data checks:
##################

print("Number of pamaters is {0}".format(autoencoder.count_params()))
pd.Series(train_data[:1000].ravel()).hist(bins=80)
plt.show()
print("Original Validation Images:")
plot_images(preprocess(train_data[:num_imgs]), nrows=int(np.sqrt(num_imgs)))
plot_model(autoencoder, to_file=os.path.join(home_dir, 'model_plot.png'),
           show_shapes=True, show_layer_names=True)
print("Validation Captions:")
print(captions[:num_imgs]["0"].values)
print("Generating Images below:")

In [ ]:
import tensorflow as tf
import keras
from keras import layers, models
from keras.layers import Input


def vstack(tensors_list):
    return tf.concat(tensors_list, axis=0)

class Dafa2_t:

    def __init__(self, denoiser, class_guidance, batch_size=64): 
        rand_image_input = keras.Input((64,64,3,))
        label_input = keras.Input((1, 512,))

        zero_array = tf.zeros((1, 512,))

        noise_in2 = tf.convert_to_tensor(np.array([0.95358411])[:, None, None, None])

        print(rand_image_input.shape, noise_in2.shape, label_input.shape)

        x1 = vstack([rand_image_input, rand_image_input])
        y1 = vstack([noise_in2,noise_in2])
        z1 = vstack([label_input, label_input])

        print(x1.shape, y1.shape, z1.shape)
        inputs = [x1,y1,z1]

        out = denoiser.call(inputs)

        self.out = out[1]

        import tf2onnx

        spec0 = tf.TensorSpec([64,64,3], tf.float32, name='input_0')
        spec1 = tf.TensorSpec([1, 512], tf.float32, name='input_1')


        input_signature = [spec0, spec1]

        onnx_model, _ = tf2onnx.convert.from_keras(self.out, input_signature=input_signature, inputs_as_nchw=[
            'input_0',
            'input_1'
        ],
                                                   opset=12
                                                   )


        # output = OriginalModelAsLayer(denoiser)(inputs, batch_size)

        self.denoiser = denoiser
        self.class_guidance = class_guidance
        self.diffusion_steps = diffusion_steps
        #TODO: parametrize this better:
        self.noise_levels = 1 - np.power(np.arange(0.0001, 0.99, 1 / self.diffusion_steps), 1 / 3)

        self.noise_levels[-1] = 0.01
        self.batch_size = batch_size
        print("noise_levels", self.noise_levels)


    def reverse_diffusion(self, seeds, label):
        """Reverse Guided Diffusion on a matrix of random images (seeds). Returns generated images"""
        print("testtttt",label.shape)

        new_img = seeds

        curr_noise, next_noise = self.noise_levels[0], self.noise_levels[1]

        # predict original denoised image:
        # x0_pred = self.predict_x_zero(new_img, label, curr_noise)
        x_t = new_img
        noise_level = curr_noise

        # we use 0 for the unconditional embedding:
        num_imgs = len(x_t)
        label_empty_ohe = np.zeros(shape=label.shape)

        # predict x0:
        noise_in = np.array([noise_level])[:, None, None, None]

        # print("noise_in2", noise_in)

        # TODO: can we do some of this in tensorflow?
        # concatenate the conditional and unconditional inputs to speed inference:

        nn_inputs = [np.vstack([x_t, x_t]),
                     np.vstack([noise_in, noise_in]),
                     np.vstack([label, label_empty_ohe])]

        x0_pred = self.denoiser.predict(nn_inputs, batch_size=self.batch_size)

        # x0_pred_label = x0_pred[:num_imgs]
        x0_pred = x0_pred[1:]

        print(x0_pred)

        return x0_pred

In [ ]:

sys.path.append("/content/guided-diffusion-keras/guided_diffusion")
from utils import batch_generator, plot_images, get_data, preprocess, slerp, get_text_encodings



prompt = "mountain landscape with cloudy sky with a house"

class_guidance = 5

diffusion_steps = 2

seed =  1

big_diffuser = Dafa2_t(autoencoder,
                        class_guidance=class_guidance,
                       )
print("hadi hadi")

print(big_diffuser.noise_levels)

num_imgs = 1
np.random.seed(seed) #- 100/120 baseline
rand_image = np.random.normal(0, 1, (num_imgs, image_size, image_size, num_channels))

prompt=[prompt]*num_imgs

labels = get_text_encodings(prompt, model)

print(len(labels[0]))

imgs = big_diffuser.reverse_diffusion(rand_image, labels)
plot_images(imgs, nrows=int(np.sqrt(num_imgs)), save_name=prompt[0], size=12)